In [132]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import time
import csv
import matplotlib.pyplot as plt
import numpy as np
#from prettytable import PrettyTable
import sqlite3


In [133]:
con = sqlite3.connect('my30.db') #создали соединение с БД
cur = con.cursor() #курсор, по строкам будет ходить

def create_table_student():
    cur.execute('CREATE TABLE IF NOT EXISTS student (vk_id INT NOT NULL, name VARCHAR(45) NOT NULL, home_town VARCHAR(45) NOT NULL, PRIMARY KEY (vk_id), CONSTRAINT fk_student_student_sex  FOREIGN KEY (name) REFERENCES student_sex (student_name) ON DELETE NO ACTION ON UPDATE NO ACTION)')

def create_table_student_sex():
    cur.execute('CREATE TABLE IF NOT EXISTS student_sex (student_name VARCHAR(45) NOT NULL, student_sex VARCHAR(45) NULL, PRIMARY KEY (student_name))')    
    

def data_entry(value):    
    cur.execute('INSERT INTO student (vk_id, name, home_town) VALUES(?, ?, ?)',value)
    con.commit()

def data_entry_sex(value):    
    cur.execute('INSERT or IGNORE INTO student_sex (student_name, student_sex) VALUES(?, ?)',value)
    con.commit()

create_table_student()
create_table_student_sex()
    
        
def user_parser (j, id, sex):
    if sex==1:
        sex1='female'
    else:
        sex1='male'  
    url = "https://api.vk.com/method/users.get.xml?user_ids="+id[1:]+"&fields=home_town"
    r = requests.get(url)
    page = BeautifulSoup(r.text, 'html.parser')
    try: #(number, url, id, home_town, name, sex)
        answer = (str(j), id, page.uid.get_text(), page.home_town.get_text(), page.first_name.get_text(), sex1)
    except AttributeError:
        answer = (str(j), id, page.uid.get_text(), 'Error', page.first_name.get_text(), sex1)
    try:
        if answer[3]!='Error':
            if answer[3]!='':
                data_entry((answer[2],answer[4],answer[3]))
                data_entry_sex((answer[4],answer[5]))
    except UnicodeEncodeError:
        answer = (str(j), id, page.uid.get_text(), 'Error', page.first_name.get_text(), sex1)
        if answer[3]!='Error':
            if answer[3]!='':
                data_entry((answer[2],answer[4],answer[3]))
                data_entry_sex((answer[4],answer[5]))
    time.sleep(0.33)

In [134]:
ua = UserAgent()
print(ua.chrome)
header = {'User-Agent':str(ua.chrome)}
print(header)

Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.16 Safari/537.36
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}


In [135]:
j=0
bdate=17

while bdate<=25:
    sex=1
    while sex<=2:
        i=1
        while i<=48:
            if i==0:
                url = "https://vk.com/search?c%5Bage_from%5D="+str(bdate)+"&c%5Bage_to%5D="+str(bdate)+"&c%5Bcity%5D=2&c%5Bcountry%5D=1&c%5Bname%5D=1&c%5Bper_page%5D=40&c%5Bphoto%5D=1&c%5Bsection%5D=people&c%5Bsex%5D="+str(sex)+"&c%5Buniversity%5D=56&offset=0"
            else:
                url = "https://vk.com/search?c%5Bage_from%5D="+str(bdate)+"&c%5Bage_to%5D="+str(bdate)+"&c%5Bcity%5D=2&c%5Bcountry%5D=1&c%5Bname%5D=1&c%5Bper_page%5D=40&c%5Bphoto%5D=1&c%5Bsection%5D=people&c%5Bsex%5D="+str(sex)+"&c%5Buniversity%5D=56&offset="+str(20+i*20)
            r=requests.get(url, headers=header)
            print(r.ok)
            page = BeautifulSoup(r.text, 'html.parser')
            #print(page.prettify())
            tags=page.findAll("div", {"class":"labeled name"})
            #print (tags)
            print('Page', i+1)
            for tag in tags:
                some=tag.findAll("a", {"class": None})
                for link in some:
                    j=j+1
                    print (j,')' ,link.get('href'))
                    user_parser(j, link.get('href'), sex)
            i=i+1
            time.sleep(0.33)
        sex=sex+1
    bdate=bdate+1
#cur.close()
#con.close()

True
Page 2
1 ) /id25676529
2 ) /snezzzhana
3 ) /nastya_stausy
4 ) /losseella
5 ) /belyakovamaria
6 ) /feykina
7 ) /lakmanito21
8 ) /id351690564
9 ) /id255127441
True
Page 3
10 ) /j_uli
11 ) /katerina.vasilyeva99
12 ) /id223057105
13 ) /id202045634
14 ) /officepluse
15 ) /arina_abramova1
16 ) /id185691616
17 ) /id184385630
18 ) /ksenmaria
19 ) /polina.chirkova
20 ) /astarto
21 ) /id145699182
22 ) /your_mom_so_hot
True
Page 4
23 ) /klassen_anna
24 ) /id98035245
25 ) /evallins
26 ) /id52029721
27 ) /maiorovavalerya
28 ) /sedovakatya
29 ) /irina.grigorieva
30 ) /id15567930
31 ) /id1116757
32 ) /id180561585
33 ) /makaroooosh
34 ) /ulchanfox
35 ) /ppppffffffffff
True
Page 5
36 ) /bascheva
37 ) /romanchuk.anastasia
38 ) /annakazmina29
39 ) /id50333890
40 ) /arbllbr
41 ) /id3837386
42 ) /mvnams
43 ) /alissd
True
Page 6
44 ) /id195225202
45 ) /yulikkkk


KeyboardInterrupt: 

In [139]:
cur.execute('UPDATE student SET home_town = "Санкт-Петербург" WHERE home_town = "Санкт - Петербург" OR "Санкт Петербург" OR "СПб" OR "Питер" OR "Saint-Petersburg" OR "Петербург" OR "Санкт-петербург" OR "Санкт- Петербург" OR "Санкт-Петербург " OR "санкт-петербург"')
con.commit()

In [140]:
cur.execute('SELECT COUNT(1) FROM student')
str_vsego = cur.fetchall()
vsego = str_vsego.pop(0)[0]
print('Число анализируемых пользователей =',vsego)

###получили списки для осей
#def graph_data():
cur.execute('SELECT home_town FROM student GROUP BY home_town')
data_town = cur.fetchall()
print(data_town)
cur.execute('SELECT COUNT(home_town) FROM student GROUP BY home_town')
data_value = cur.fetchall()
print(data_value)   
towns = []
values = []
values_percent=[]
b = 0
while b<len(data_town):
    towns.append(data_town.pop(b)[0])
    values.append(data_value.pop(b)[0])
    values_percent.append(round(values.pop(b)*100/vsego, 2))
    
print(towns)
print(values_percent)
###
#cur.execute('SELECT COUNT(1) FROM student_sex WHERE student_sex = "male"')
#datam = cur.fetchall()   
#cur.execute('SELECT COUNT(1) FROM student_sex WHERE student_sex = "female"')
#dataf = cur.fetchall()
#print('boys =',datam.pop(0)[0],'girls =',dataf.pop(0)[0]) 


Число анализируемых пользователей = 14
[('Актюбинск',), ('Александров',), ('Владивосток',), ('Волгоград',), ('Лянтор',), ('Москва',), ('Орск',), ('Рязань',), ('С-Петербург',), ('Сургут',), ('печоры',)]
[(1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (4,), (1,), (1,)]
['Актюбинск', 'Александров', 'Владивосток', 'Волгоград', 'Лянтор', 'Москва', 'Орск', 'Рязань', 'С-Петербург', 'Сургут', 'печоры']
[7.14, 7.14, 7.14, 7.14, 7.14, 7.14, 7.14, 7.14, 28.57, 7.14, 7.14]


In [128]:
#пока не строится
fig = plt.figure()
axes = fig.add_subplot (1, 1, 1)
plt.barh(towns[:], values_percent[:], [1,2], align='center', color='orange')
axes.set_xscale ('linear')
plt.title('Распределение студентов в %')
plt.grid(True)   # линии вспомогательной сетки
plt.show()

ValueError: could not convert string to float: 'pkjihb'